In [1]:
import tensorflow as tf


In [3]:
"""加载数据集合"""
# 定义训练数据的文件路径
train_dir = 'D:/Paper reproduction/RiceNet A deep convolutional neural network approach for classification of rice varieties/Dataset/items_train_val_set/train'
valid_dir = 'D:/Paper reproduction/RiceNet A deep convolutional neural network approach for classification of rice varieties/Dataset/items_train_val_set/validation'
dvi_valid_dir = 'D:/Paper reproduction/RiceNet A deep convolutional neural network approach for classification of rice varieties/Dataset/items_val_test_set/valid'
# 定义批次大小
batch_size = 32
# 定义图像大小
image_size = (512,512)

# 配置训练数据集合的数据增强
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,         # 图像进行归一化处理
    rotation_range=20,      # 旋转角范围 0~20
    width_shift_range=0.2,  # 水平随机移动范围
    height_shift_range=0.2, # 垂直随机移动范围
    shear_range=0.2,        # 剪切强度
    zoom_range=0.2,         # 随机缩放的范围
    horizontal_flip=True    # 是否随机水平翻转
)
train_generator = train_datagen.flow_from_directory(
        train_dir,                # train 数据集合 的文件路径
        target_size=image_size,   # 输出的图像的大小
        batch_size=batch_size,    # 输出的批次大小
        class_mode='categorical') # 选取分类标签的形式 ， 这里是独热编码 (因为loss是交叉熵损失函数 categorical_crossentropy)

# 配置测试数据集合的数据增强
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        dvi_valid_dir,                # train 数据集合 的文件路径
        target_size=image_size,   # 输出的图像的大小
        batch_size=batch_size,    # 输出的批次大小
        class_mode='categorical') # 选取分类标签的形式 ， 这里是独热编码 (因为loss是交叉熵损失函数 categorical_crossentropy)

# 查看标签分配的类别对应的索引
train_generator.class_indices , test_generator.class_indices


Found 3430 images belonging to 5 classes.
Found 735 images belonging to 5 classes.


({'jehlum': 0, 'mushkbudji': 1, 'sr-1': 2, 'sr-2': 3, 'sr-4': 4},
 {'jehlum': 0, 'mushkbudji': 1, 'sr-1': 2, 'sr-2': 3, 'sr-4': 4})

In [3]:
import os
"""找出一共需要多少次批次 tatal_sample = batch_size * num_steps"""

total_class = os.listdir(train_dir)
print('共有 {} 个类别 , 分别是 ： {}'.format(len(total_class),total_class))

total_train_sample_number = 0
for signel_class in total_class:
    path = os.path.join(train_dir,signel_class)
    total_train_sample_number += len(os.listdir(path))
print('训练样本的总数：{}'.format(total_train_sample_number))

total_validation_sample_number = 0
for signel_class in total_class:
    path = os.path.join(valid_dir,signel_class)
    total_validation_sample_number += len(os.listdir(path))
print('测试集合和验证集合样本的总数：{}'.format(total_validation_sample_number))


共有 5 个类别 , 分别是 ： ['jehlum', 'mushkbudji', 'sr-1', 'sr-2', 'sr-4']
训练样本的总数：3430
测试集合和验证集合样本的总数：1470


In [4]:
# 定义模型
def RiceNet_Block(Conv2D_filters, Conv2D_kernel_size, Conv2D_strides, Conv2D_padding, Activation = 'relu'):
    return tf.keras.Sequential([
        tf.keras.layers.Conv2D(filters=Conv2D_filters, kernel_size = Conv2D_kernel_size, strides = Conv2D_strides, padding = Conv2D_padding, use_bias=True),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation(Activation)
    ])

def RiceNet():
    RiceNet = tf.keras.Sequential([
        # Input layer
        tf.keras.layers.InputLayer(input_shape = (512,512,3)),

        # big block 1
        RiceNet_Block(Conv2D_filters = 16, Conv2D_kernel_size = (5,5), Conv2D_strides = (1,1), Conv2D_padding = 'same', Activation = 'relu'),
        # big block 2
        RiceNet_Block(Conv2D_filters = 16, Conv2D_kernel_size = (5,5), Conv2D_strides = (1,1), Conv2D_padding = 'same', Activation = 'relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='valid'),
        # big block 3
        RiceNet_Block(Conv2D_filters = 32, Conv2D_kernel_size = (3,3), Conv2D_strides = (1,1), Conv2D_padding = 'same', Activation = 'relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='valid'),
        # big block 4
        RiceNet_Block(Conv2D_filters = 32, Conv2D_kernel_size = (3,3), Conv2D_strides = (1,1), Conv2D_padding = 'same', Activation = 'relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='valid'),
        # big block 5
        RiceNet_Block(Conv2D_filters = 64, Conv2D_kernel_size = (3,3), Conv2D_strides = (1,1), Conv2D_padding = 'same', Activation = 'relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='valid'),
        # big block 6
        RiceNet_Block(Conv2D_filters = 64, Conv2D_kernel_size = (3,3), Conv2D_strides = (1,1), Conv2D_padding = 'same', Activation = 'relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='valid'),
         # big block 7
        RiceNet_Block(Conv2D_filters = 128, Conv2D_kernel_size = (3,3), Conv2D_strides = (1,1), Conv2D_padding = 'same', Activation = 'relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='valid'),
        # big block 8
        RiceNet_Block(Conv2D_filters = 128, Conv2D_kernel_size = (3,3), Conv2D_strides = (1,1), Conv2D_padding = 'same', Activation = 'relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='valid'),
        # big block 9
        RiceNet_Block(Conv2D_filters = 192, Conv2D_kernel_size = (3,3), Conv2D_strides = (1,1), Conv2D_padding = 'same', Activation = 'relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='valid'),
        # big block 10
        RiceNet_Block(Conv2D_filters = 192, Conv2D_kernel_size = (3,3), Conv2D_strides = (1,1), Conv2D_padding = 'same', Activation = 'relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='valid'),

        # FC layer
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units = 192),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.Dropout(0.5),

        # Output layer
        tf.keras.layers.Dense(units = 5),
        tf.keras.layers.Softmax()
    ])

    return RiceNet

RiceNet = RiceNet()
# 查看模型结构
RiceNet.summary()


Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 512, 512, 16)      1280      
                                                                 
 sequential_1 (Sequential)   (None, 512, 512, 16)      6480      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 256, 256, 16)     0         
 )                                                               
                                                                 
 sequential_2 (Sequential)   (None, 256, 256, 32)      4768      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 128, 128, 32)     0         
 2D)                                                             
                                                                 
 sequential_3 (Sequential)   (None, 128, 128, 32)    

In [5]:
# 训练
lr = 0.00003
num_epochs = 50
# 配置模型
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
RiceNet.compile(optimizer=optimizer,loss=loss,metrics = ['accuracy'])
# 训练模型
RiceNet.fit(train_generator,epochs = num_epochs, verbose = 2, steps_per_epoch = total_train_sample_number // batch_size, validation_steps = total_validation_sample_number // batch_size, batch_size = batch_size)


OSError: unrecognized data stream contents when reading image file

In [4]:
# transfer model of  InceptionV3

# load pre-training model
pre_training_InceptionV3 = tf.keras.applications.inception_v3.InceptionV3(
    include_top = False,
    weights = 'imagenet',
    input_shape = (512,512,3),
    pooling = None,
)



ValueError: When setting `include_top=True` and loading `imagenet` weights, `input_shape` should be (299, 299, 3).  Received: input_shape=(512, 512, 3)

In [ ]:
# transfer model of InceptionResNetV2

# load pre-training model
pre_training_InceptionResNetV2 = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(
    include_top = False,
    weights = 'imagenet',
    input_shape = (512,512,3),
    pooling = None,
)
#